In [29]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler


class PcaInputter:
    def __init__(self, data):
        
        if not isinstance(data, (pd.DataFrame, np.ndarray)):
            raise Exception("Input data must be a pandas DataFrame or a numpy array.")
        
        if isinstance(data, pd.DataFrame):
            if not all(data.applymap(np.isreal).all()):
                raise Exception("All values in the DataFrame must be numerical.")
            
            if data.apply(lambda row: row.isnull().all(), axis=1).any():
                raise Exception("Found a row where all columns are NaN in DataFrame.")
        
            if data.apply(lambda col: col.isnull().all(), axis=0).any():
                raise Exception("Found a column where all rows are NaN in DataFrame.")
            
            else:
                self.r_idx, self.c_idx = np.where(data.isnull())
                #self.raw_data = data.values               
        
        if isinstance(data, np.ndarray):
            if not np.isreal(data).all():
                raise Exception("All values in the numpy array must be numerical.")
                
            if np.all(np.isnan(data), axis=1).any():
                raise Exception("Found a row where all columns are NaN in numpy array.")
        
            if np.all(np.isnan(data), axis=0).any():
                raise Exception("Found a column where all rows are NaN in numpy array.")
    
            else:
                self.r_idx, self.c_idx = np.where(np.isnan(data))
                #self.raw_data = data
                
        #scaler = StandardScaler(with_std=True,with_mean=True)
        #self.na_data = scaler.fit_transform(data)
        self.na_data = data
        

    def run_pca(self, X, M=1):
        pca_obj = PCA().fit(X)
        self.scores = pca_obj.transform(X)
        self.components = pca_obj.components_

        return self.scores[:,1-M].reshape(50,M) @self.components[:M][0].reshape(M,4)
        #return L.dot(components[:M,:])
        
    def low_rank(self,X, M=1):
        U, D, V = np.linalg.svd(X)
        L = U[:,:M] * D[None,:M]
        return L.dot(V[:M])
    
    
    def iterfill(self):
        hat_data = self.na_data.copy()
        bar_data = np.nanmean(hat_data, axis=0)
        hat_data[self.r_idx, self.c_idx] = bar_data[self.c_idx]
        
        thresh = 1e-7
        rel_err = 1
        count = 0
        ismiss = np.isnan(self.na_data)
        mssold = np.mean(hat_data[~ismiss]**2)
        mss0 = np.mean(self.na_data[~ismiss]**2)
        mssold

        while rel_err > thresh:
            count += 1
            app_data = self.run_pca(hat_data, M=1)
            hat_data[ismiss] = app_data[ismiss]
            mss = np.mean(((self.na_data - app_data)[~ismiss])**2)
            rel_err = (mssold - mss) / mss0
            mssold = mss
            #print(mssold,mss,mss0)
            print("Iteration: {0}, MSS:{1:.3f}, Rel.Err {2:.2e}".format(count, mss, rel_err))
        return hat_data
        
        

In [30]:

url = "https://raw.githubusercontent.com/JWarmenhoven/ISLR-python/master/Notebooks/Data/USArrests.csv"

USArrests = pd.read_csv(url)
USArrests = USArrests.set_index('Unnamed: 0')


X = USArrests.values

n_omit = 20
np.random.seed(15)
ridx = np.random.choice(np.arange(X.shape[0]), n_omit, replace=False)
cidx = np.random.choice(np.arange(X.shape[1]), n_omit, replace=True)
Xna = X.copy()
Xna[ridx, cidx] = np.nan

scaler = StandardScaler(with_std=True, with_mean=True)
Xna = scaler.fit_transform(Xna)


inputter_object = PcaInputter(Xna)
Xinputted = inputter_object.iterfill()

print(Xna)
print("/n")
print(Xinputted)

Iteration: 1, MSS:0.406, Rel.Err 5.94e-01
Iteration: 2, MSS:0.395, Rel.Err 1.17e-02
Iteration: 3, MSS:0.394, Rel.Err 1.12e-03
Iteration: 4, MSS:0.394, Rel.Err 1.32e-04
Iteration: 5, MSS:0.394, Rel.Err 1.79e-05
Iteration: 6, MSS:0.394, Rel.Err 2.71e-06
Iteration: 7, MSS:0.394, Rel.Err 4.39e-07
Iteration: 8, MSS:0.394, Rel.Err 7.45e-08
[[ 1.33568755  0.6762877  -0.56219529 -0.04739163]
 [ 0.5482385   1.00025132 -1.2724958   2.4064416 ]
 [ 0.08069064  1.37220955  1.00046582  0.98469273]
 [ 0.25294511  0.12434967 -1.1304357          nan]
 [ 0.30216068  1.1562338   1.78179638  1.99571415]
 [ 0.03147507  0.29233081  0.85840572  1.79561616]
 [-1.10048293 -0.83554254  0.78737567 -1.11107041]
 [-0.46068058  0.700285    0.43222541 -0.61609117]
 [ 1.87705876  1.86415431  1.00046582  1.07947599]
 [        nan  0.37632138 -0.42013519  0.43705613]
 [-0.60832728 -1.60345631  1.21355597 -0.15270636]
 [-1.2727374  -0.71555601 -0.8463155  -0.78459474]
 [ 0.64666963  0.83227018  1.21355597  0.24748962]
 

In [ ]:




Xhat = Xna.copy()
Xbar = np.nanmean(Xhat, axis=0)
Xhat[r_idx, c_idx] = Xbar[c_idx]



In [ ]:
#Xhat[~ismiss]

In [ ]:
def run_pca(X, M=1):
    pca_obj = PCA().fit(X)
    scores = pca_obj.transform(X)
    components = pca_obj.components_

    return scores[:,1-M].reshape(50,M) @components[:M][0].reshape(M,4)

run_pca(USArrests)
USArrests

In [ ]:
np.corrcoef(Xapp[ismiss], X[ismiss])[0,1]

In [ ]:
hat_data = self.na_data.copy()
bar_data = np.nanmean(hat_data, axis=0)
hat_data[self.r_idx, self.c_idx] = bar_data[self.c_idx]

## Cells

In [ ]:
import pandas as pd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.datasets import get_rdataset
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

url = "https://raw.githubusercontent.com/JWarmenhoven/ISLR-python/master/Notebooks/Data/USArrests.csv"

USArrests = pd.read_csv(url)
USArrests = USArrests.set_index('Unnamed: 0')

scaler = StandardScaler(with_std=True,with_mean=True)
USArrests = scaler.fit_transform(USArrests)

In [ ]:
X=USArrests#.values
n_omit = 20
np.random.seed(15)
r_idx = np.random.choice(np.arange(X.shape[0]),n_omit,replace=False)
c_idx = np.random.choice(np.arange(X.shape[1]),n_omit,replace=True)

Xna = X.copy()
Xna[r_idx, c_idx] = np.nan

In [ ]:
def low_rank(X, M=1):
    U, D, V = np.linalg.svd(X)
    L = U[:,:M] * D[None,:M]
    return L.dot(V[:M])

Xhat = Xna.copy()
Xbar = np.nanmean(Xhat, axis=0)
Xhat[r_idx, c_idx] = Xbar[c_idx]

thresh = 1e-7
rel_err = 1
count = 0
ismiss = np.isnan(Xna)
mssold = np.mean(Xhat[~ismiss]**2)
mss0 = np.mean(Xna[~ismiss]**2)

while rel_err > thresh:
    count += 1
    Xapp = low_rank(Xhat, M=1)
    Xhat[ismiss] = Xapp[ismiss]
    mss = np.mean(((Xna - Xapp)[~ismiss])**2)
    rel_err = (mssold - mss) / mss0
    mssold = mss
    print("Iteration: {0}, MSS:{1:.3f}, Rel.Err {2:.2e}".format(count, mss, rel_err))
Xhat

In [ ]:
X=USArrests_scaled
U, D, V = np.linalg.svd(X, full_matrices=False)
U.shape, D.shape, V.shape
L = U[:,:M] * D[None,:M]

def low_rank(X, M=1):
    U, D, V = np.linalg.svd(X)
    L = U[:,:M] * D[None,:M]
    return L.dot(V[:M])